In [ ]:
%matplotlib inline

import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from datetime import datetime
from graph_tool.all import * 

In [ ]:
#G = load_graph_from_csv("data/data.txt", directed=True, eprop_types=['int', 'int'], eprop_names=['start', 'end'], string_vals=False, csv_options={'delimiter':' '})
#G.save('data/original.gt')

In [ ]:
G = load_graph('data/original.gt')

In [ ]:
graphs = []
for name in os.listdir('data/snapshots/csv'):
    g = load_graph_from_csv(os.path.join('data/snapshots/csv', name), directed=True, string_vals=False)
    timestamp = g.new_graph_property('python::object')
    timestamp[g] = datetime.fromtimestamp(int(name[:-4]))
    g.graph_properties['timestamp'] = timestamp
    graphs.append(g)

In [ ]:
for g in graphs:
    g.vertex_properties['pagerank'] = pagerank(g)

In [ ]:
#for g in graphs:
    #g.vertex_properties['betweenness'], g.edge_properties['betweenness'] = betweenness(g) SLOW

In [ ]:
for g in graphs:
    hits_eigen = g.new_graph_property('float')
    hits_eigen[g], g.vertex_properties['hits_authority'], g.vertex_properties['hits_hub'] = hits(g)
    g.graph_properties['hits_eigen'] = hits_eigen

In [ ]:
for g in graphs:
    degree = g.new_graph_property('python::object')
    d = {}
    d['total'], d['total_std'] = vertex_average(g, 'total')
    d['in'], d['in_std'] = vertex_average(g, 'in')
    d['out'], d['out_std'] = vertex_average(g, 'out')
    degree[g] = d
    g.graph_properties['degree'] = degree

In [ ]:
for g in graphs:
    clustering_coef = g.new_graph_property('python::object')
    c = {}
    c['value'], c['std'] = global_clustering(g)
    clustering_coef[g] = c
    g.graph_properties['clustering'] = clustering_coef

In [ ]:
for g in graphs:
    diameter = g.new_graph_property('float')
    diameter[g], _ = pseudo_diameter(g)
    g.graph_properties['diameter'] = diameter

In [ ]:
for g in graphs:
    g.vertex_properties['largest_connected_component'] = label_largest_component(g)

In [ ]:
#for g in graphs:
    #g.vertex_properties['shortest_distances'] = shortest_distance(g) SLOW

In [ ]:
#for g in graphs:
    # TODO: similarity(g1, g2) or vertex_similarity(g1, g2) ??

In [ ]:
#graph_draw(graphs[0], vertex_fill_color=g.vertex_properties['pagerank'], output="pagerank_0.pdf") SLOW???

In [ ]:
d = {}
for g in graphs:
    for k, v in g.graph_properties.items():
        v=v[g]
        if isinstance(v, dict):
            for sk, sv in v.items():
                sk = k+'_'+sk
                if not sk in d:
                    d[sk] = []
                    
                d[sk].append(sv)
        else:
            if not k in d:
                d[k] = []
                
            d[k].append(v)
        
df = pd.DataFrame(d)
display(df)